## 문제 정의
- 통신사의 고객 유지를 위해 고객 이탈을 예측
- 고객의 개인정보및 가입정보를 입력으로 하여 이탈 여부를 분류하는 이진 분류 문제
    - 평가지표: AUC

- 17시 40분 까지 제출횟수 제한 없이 저한테 예측한 csv 파일 전달해주면 됩니다.
    - 많이 제출할 수록 유리함!!! 많이 제출해 볼 것!!
- 1등과 2등한테 스타벅스 커피쿠폰 줄게요!(개인 사비로!!!!!)
    



- 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 데이터 경로 변수


In [2]:
DATA_PATH = "/content/drive/MyDrive/data/"
DATA_PATH

'/content/drive/MyDrive/data/'

- 시드값

In [3]:
SEED = 42

- 데이터 불러오기

In [4]:
import pandas as pd
import numpy as np

train = pd.read_csv(f"{DATA_PATH}customer_train.csv") # 학습데이터
test = pd.read_csv(f"{DATA_PATH}customer_test.csv") # 테스트 데이터
train.shape , test.shape

((4930, 21), (2113, 20))

# 통신사 고객 이탈 여부 데이터셋 설명
- 이탈 여부 컬럼이 타겟으로 1이 이탈!
- 성별 컬럼
    - 1 = 여성, 0 = 남성
- 여부 관련 컬럼
    - 1 = yes, 0 = no
- 약정기간 컬럼
    - 0 = 약정 없음
    - 1 = 1년 약정, 2 = 2년 약정

In [5]:
train.head()

,ID,성별,고령자여부,기혼여부,부양가족여부,가입기간,집전화이용여부,다중회선여부,인터넷이용방식,인터넷보안서비스사용여부,...,기기방화벽서비스사용여부,인터넷기술지원서비스사용여부,스트리밍TV여부,스트리밍영화여부,약정기간,지로여부,지불방법,한달요금,총지불요금,이탈여부
0,train_0,1,0,0,0,23,1,0,광,0,...,0,1,0,0,0,0,메일,79.10,1783.75,0
1,train_1,1,0,0,0,1,1,0,디지털,0,...,0,0,0,0,0,0,메일,45.40,45.40,1
2,train_2,0,0,0,0,23,1,1,광,0,...,1,1,1,1,0,0,메일,104.05,2470.10,1
3,train_3,1,0,0,0,11,1,0,디지털,0,...,1,1,0,1,1,1,신용카드,64.90,697.25,0
4,train_4,1,0,0,0,5,1,1,광,0,...,0,0,1,0,0,0,전자,85.20,474.80,1


# 데이터 분석(Data analysis)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4930 entries, 0 to 4929
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              4930 non-null   object 
 1   성별              4930 non-null   int64  
 2   고령자여부           4930 non-null   int64  
 3   기혼여부            4930 non-null   int64  
 4   부양가족여부          4930 non-null   int64  
 5   가입기간            4930 non-null   int64  
 6   집전화이용여부         4930 non-null   int64  
 7   다중회선여부          4930 non-null   int64  
 8   인터넷이용방식         3859 non-null   object 
 9   인터넷보안서비스사용여부    4930 non-null   int64  
 10  인터넷백업서비스사용여부    4930 non-null   int64  
 11  기기방화벽서비스사용여부    4930 non-null   int64  
 12  인터넷기술지원서비스사용여부  4930 non-null   int64  
 13  스트리밍TV여부        4930 non-null   int64  
 14  스트리밍영화여부        4930 non-null   int64  
 15  약정기간            4930 non-null   int64  
 16  지로여부            4930 non-null   int64  
 17  지불방법            4930 non-null   o

- 결측치 처리하기

In [7]:
train.isnull().sum()

ID                   0
성별                   0
고령자여부                0
기혼여부                 0
부양가족여부               0
가입기간                 0
집전화이용여부              0
다중회선여부               0
인터넷이용방식           1071
인터넷보안서비스사용여부         0
인터넷백업서비스사용여부         0
기기방화벽서비스사용여부         0
인터넷기술지원서비스사용여부       0
스트리밍TV여부             0
스트리밍영화여부             0
약정기간                 0
지로여부                 0
지불방법                 0
한달요금                 0
총지불요금                0
이탈여부                 0
dtype: int64

In [8]:
test.isnull().sum()

ID                  0
성별                  0
고령자여부               0
기혼여부                0
부양가족여부              0
가입기간                0
집전화이용여부             0
다중회선여부              0
인터넷이용방식           455
인터넷보안서비스사용여부        0
인터넷백업서비스사용여부        0
기기방화벽서비스사용여부        0
인터넷기술지원서비스사용여부      0
스트리밍TV여부            0
스트리밍영화여부            0
약정기간                0
지로여부                0
지불방법                0
한달요금                0
총지불요금               0
dtype: int64

In [9]:
train["인터넷이용방식"] = train["인터넷이용방식"].fillna("UNK")
test["인터넷이용방식"] = test["인터넷이용방식"].fillna("UNK")

In [10]:
train.isnull().sum().sum() , test.isnull().sum().sum()

(0, 0)

## 타겟값 확인
- 이탈 비율 확인하기

In [11]:
train["이탈여부"].mean()

0.2653144016227181

In [12]:
train.columns

Index(['ID', '성별', '고령자여부', '기혼여부', '부양가족여부', '가입기간', '집전화이용여부', '다중회선여부',
       '인터넷이용방식', '인터넷보안서비스사용여부', '인터넷백업서비스사용여부', '기기방화벽서비스사용여부',
       '인터넷기술지원서비스사용여부', '스트리밍TV여부', '스트리밍영화여부', '약정기간', '지로여부', '지불방법',
       '한달요금', '총지불요금', '이탈여부'],
      dtype='object')

# 특성 공학(Feature Engineering)
- Feature Engineering 과정에서 평가를 위한 예측을 해야하기 때문에 테스트 세트에 대해서도 동일한 작업을 진행해줘야 한다.


## Feature Extraction

- 특성으로 사용할 변수 추가하기

In [15]:
train_ft = train.iloc[:,1:-1].copy()
test_ft = test.iloc[:,1:].copy()
train_ft.shape, test_ft.shape

((4930, 19), (2113, 19))

## Feature Encoding

- One-Hot Encoding

In [16]:
from sklearn.preprocessing import OneHotEncoder
cols = ["인터넷이용방식","지불방법"]
enc = OneHotEncoder(handle_unknown = 'ignore')

In [17]:
# 학습데이터
tmp = pd.DataFrame(
    enc.fit_transform(train_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols)
train_ft.head()

,성별,고령자여부,기혼여부,부양가족여부,가입기간,집전화이용여부,다중회선여부,인터넷보안서비스사용여부,인터넷백업서비스사용여부,기기방화벽서비스사용여부,...,지로여부,한달요금,총지불요금,인터넷이용방식_UNK,인터넷이용방식_광,인터넷이용방식_디지털,지불방법_메일,지불방법_신용카드,지불방법_자동이체,지불방법_전자
0,1,0,0,0,23,1,0,0,1,0,...,0,79.10,1783.75,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1,0,0,0,1,1,0,0,0,0,...,0,45.40,45.40,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,0,0,0,0,23,1,1,0,0,1,...,0,104.05,2470.10,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,1,0,0,0,11,1,0,0,0,1,...,1,64.90,697.25,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1,0,0,0,5,1,1,0,0,0,...,0,85.20,474.80,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [18]:
# 테스트데이터
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols)
test_ft.head()

,성별,고령자여부,기혼여부,부양가족여부,가입기간,집전화이용여부,다중회선여부,인터넷보안서비스사용여부,인터넷백업서비스사용여부,기기방화벽서비스사용여부,...,지로여부,한달요금,총지불요금,인터넷이용방식_UNK,인터넷이용방식_광,인터넷이용방식_디지털,지불방법_메일,지불방법_신용카드,지불방법_자동이체,지불방법_전자
0,0,0,0,0,8,1,0,0,0,1,...,0,94.70,740.30,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1,0,0,0,52,1,0,1,0,0,...,0,49.75,2535.55,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0,0,1,0,20,1,0,0,0,0,...,0,89.00,1820.45,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0,1,1,0,70,1,0,1,1,1,...,1,105.35,7511.90,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,1,0,0,0,1,1,0,1,0,0,...,0,48.45,48.45,0.0,0.0,1.0,0.0,0.0,1.0,0.0


## Feature Scaling
- Min-Max Scaling
    

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [20]:
# 학습데이터
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
train_ft.head()

,성별,고령자여부,기혼여부,부양가족여부,가입기간,집전화이용여부,다중회선여부,인터넷보안서비스사용여부,인터넷백업서비스사용여부,기기방화벽서비스사용여부,...,지로여부,한달요금,총지불요금,인터넷이용방식_UNK,인터넷이용방식_광,인터넷이용방식_디지털,지불방법_메일,지불방법_신용카드,지불방법_자동이체,지불방법_전자
0,1.0,0.0,0.0,0.0,0.319444,1.0,0.0,0.0,1.0,0.0,...,0.0,0.606076,0.205388,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.013889,1.0,0.0,0.0,0.0,0.0,...,0.0,0.270418,0.005228,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.319444,1.0,1.0,0.0,0.0,1.0,...,0.0,0.854582,0.284416,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.152778,1.0,0.0,0.0,0.0,1.0,...,1.0,0.464641,0.080284,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.069444,1.0,1.0,0.0,0.0,0.0,...,0.0,0.666833,0.054670,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [21]:
# 테스트 데이터
test_ft[test_ft.columns] = scaler.transform(test_ft)
test_ft.head()

,성별,고령자여부,기혼여부,부양가족여부,가입기간,집전화이용여부,다중회선여부,인터넷보안서비스사용여부,인터넷백업서비스사용여부,기기방화벽서비스사용여부,...,지로여부,한달요금,총지불요금,인터넷이용방식_UNK,인터넷이용방식_광,인터넷이용방식_디지털,지불방법_메일,지불방법_신용카드,지불방법_자동이체,지불방법_전자
0,0.0,0.0,0.0,0.0,0.111111,1.0,0.0,0.0,0.0,1.0,...,0.0,0.761454,0.085241,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.722222,1.0,0.0,1.0,0.0,0.0,...,0.0,0.313745,0.291953,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.277778,1.0,0.0,0.0,0.0,0.0,...,0.0,0.704681,0.209613,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.972222,1.0,0.0,1.0,1.0,1.0,...,1.0,0.867530,0.864948,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.013889,1.0,0.0,1.0,0.0,0.0,...,0.0,0.300797,0.005579,0.0,0.0,1.0,0.0,0.0,1.0,0.0


- 정답 데이터

In [22]:
target = train["이탈여부"]
target

0       0
1       1
2       1
3       0
4       1
       ..
4925    0
4926    0
4927    0
4928    0
4929    0
Name: 이탈여부, Length: 4930, dtype: int64

In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=5,shuffle=True, random_state=SEED)

# 모델 학습및 교차 검증

In [24]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=SEED)
scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='roc_auc',n_jobs = -1)
np.mean(scores)

0.843274999975813

# 모델 학습


In [25]:
model = LogisticRegression(random_state=SEED)
model.fit(train_ft,target)

LogisticRegression(random_state=42)

# 테스트 데이터 예측


In [26]:
pred = model.predict_proba(test_ft)[:,1]
pred

array([0.72828999, 0.07643314, 0.61312863, ..., 0.00442743, 0.01609077,
       0.63223254])

# 평가를 위한 제출 파일 생성


In [27]:
submit = pd.DataFrame(pred, columns = ["target"])
submit

,target
0,0.728290
1,0.076433
2,0.613129
3,0.042137
4,0.344026
...,...
2108,0.636313
2109,0.053044
2110,0.004427
2111,0.016091


- 예측 결과를 csv 파일로 저장하여 제출

In [32]:
submit.to_csv("홍길동3.csv",index=False)